In [116]:
from importlib import reload

#%reset -f
import s_iForest_functions
reload(s_iForest_functions)
from s_iForest_functions import *

In [117]:
import pandas as pd 

# Data to be used for the DataFrame
data = {
    'ipv4': ['56.73.174.168', '56.73.174.168', '56.73.174.168', '33.73.174.111', '33.73.174.111', '33.73.174.111'],
    #'ipv4': ['1', '2', '3', '4', '5', '6'],
    'scan_id': [0, 1, 2, 3, 4, 5],
    'port': [[161, 22, 25, 111, 110],[161, 33, 22, 25, 110],[161, 22, 25, 110], [22, 110, 33, 25],[22, 110, 45, 25],[22, 110, 25]],
    'service': [['SNMP', 'SSH', 'SMTP', 'POP3', 'HTTP'],['POP3', 'SNMP', 'SSH', 'HTTP', 'SMTP'],['SNMP', 'SMTP', 'HTTP' 'POP3'], ['SSH', 'HSS', 'POP3', 'SMTP'],['SSH', 'POP3', 'X', 'SMTP'],['SSH', 'POP3', 'SMTP']]
}

# Creating the DataFrame
#df = pd.DataFrame(data)

# Displaying the DataFrame
#print(df)


In [118]:
import pandas as pd
import numpy as np
import random
import os
import re
from collections import defaultdict


# Example function to generate the service_port_dict
def generate_service_port_dict(directory_path):
    service_port_dict = defaultdict(lambda: defaultdict(int))
    pattern = re.compile(r'(\d+)/([A-Z0-9]+)')

    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r') as file:
                for line in file:
                    matches = pattern.findall(line)
                    for port, service in matches:
                        service_port_dict[service][port] += 1
    return service_port_dict

# Calculate distributions
def calculate_distributions(service_port_dict):
    total_services = sum(sum(ports.values()) for ports in service_port_dict.values())
    service_distribution = {service: sum(ports.values()) / total_services for service, ports in service_port_dict.items()}

    port_distributions = {}
    for service, ports in service_port_dict.items():
        total_ports = sum(ports.values())
        port_distributions[service] = {int(port): count / total_ports for port, count in ports.items()}
    
    return service_distribution, port_distributions

# Function to generate scan data
def generate_scan_data(num_ips, num_scans_per_ip, norm_list_length_distribution, service_distribution, port_distributions, anom_list_length_distribution, anom_service_distribution, anom_port_distribution, anomaly_rate=0.1):
    def generate_ipv4():
        return ".".join(map(str, (random.randint(0, 255) for _ in range(4))))

    data = []
    global_scan_id = 0  # Initialize a global scan ID counter

    for _ in range(num_ips):
        ipv4 = generate_ipv4()
        # Determine if this IP will exhibit anomalous behavior
        is_anomalous_ip = np.random.rand() < anomaly_rate
        
        if is_anomalous_ip:
            list_length_distribution = anom_list_length_distribution
            service_dist = anom_service_distribution
            port_dist = anom_port_distribution
        else:
            list_length_distribution = norm_list_length_distribution
            service_dist = service_distribution
            port_dist = port_distributions
        
        # Pre-select services and ports for this IP
        list_length = np.random.choice(np.arange(1, len(list_length_distribution) + 1), p=list_length_distribution)
        services_chosen = np.random.choice(list(service_dist.keys()), list_length, p=list(service_dist.values()), replace=False)
        ports_chosen = {service: np.random.choice(list(port_dist[service].keys()), p=list(port_dist[service].values())) for service in services_chosen}

        for _ in range(num_scans_per_ip):
            if is_anomalous_ip:
                # Introduce anomalies by changing the behavior
                current_length = np.random.choice(np.arange(1, len(anom_list_length_distribution) + 1), p=anom_list_length_distribution)
                current_services = np.random.choice(list(anom_service_distribution.keys()), current_length, p=list(anom_service_distribution.values()), replace=False)
                current_ports = [np.random.choice(list(anom_port_distribution[service].keys()), p=list(anom_port_distribution[service].values())) for service in current_services]
            else:
                current_length = random.randint(1, list_length)
                current_services = random.sample(services_chosen.tolist(), current_length)
                current_ports = [ports_chosen[service] for service in current_services]

            anomaly = 1 if is_anomalous_ip else 0
            data.append([ipv4, global_scan_id, current_ports, current_services, anomaly])
            global_scan_id += 1  # Increment the global scan ID

    return pd.DataFrame(data, columns=['ipv4', 'scan_id', 'port', 'service', 'anomaly'])

# Example directory path
directory_path = '/Users/christiedjidjev/Censys Pages'
service_port_dict = generate_service_port_dict(directory_path)

# Calculate distributions
norm_service_distribution, norm_port_distributions = calculate_distributions(service_port_dict)

# Reasonable test values
num_ips = 2
num_scans_per_ip = 20
norm_list_length_distribution = [0.1, 0.2, 0.3, 0.2, 0.1, 0.05, 0.05]  # probabilities for list lengths 1 through 7
anom_list_length_distribution = [0,0,0,0,0,0,0,0.05, 0.1, 0.2, 0.3, 0.2, 0.1, 0.05]  # different probabilities for anomalous behavior
anom_service_distribution = norm_service_distribution  # use the same service distribution for anomalies
anom_port_distribution = norm_port_distributions  # use the same port distribution for anomalies
anomaly_rate = 0.2  # 20% of the IPs will exhibit anomalous behavior

# Generate the data
df = generate_scan_data(num_ips, num_scans_per_ip, norm_list_length_distribution, norm_service_distribution, norm_port_distributions, anom_list_length_distribution, anom_service_distribution, anom_port_distribution, anomaly_rate)
print(df)

              ipv4  scan_id  \
0   176.155.49.134        0   
1   176.155.49.134        1   
2   176.155.49.134        2   
3   176.155.49.134        3   
4   176.155.49.134        4   
5   176.155.49.134        5   
6   176.155.49.134        6   
7   176.155.49.134        7   
8   176.155.49.134        8   
9   176.155.49.134        9   
10  176.155.49.134       10   
11  176.155.49.134       11   
12  176.155.49.134       12   
13  176.155.49.134       13   
14  176.155.49.134       14   
15  176.155.49.134       15   
16  176.155.49.134       16   
17  176.155.49.134       17   
18  176.155.49.134       18   
19  176.155.49.134       19   
20  241.136.198.77       20   
21  241.136.198.77       21   
22  241.136.198.77       22   
23  241.136.198.77       23   
24  241.136.198.77       24   
25  241.136.198.77       25   
26  241.136.198.77       26   
27  241.136.198.77       27   
28  241.136.198.77       28   
29  241.136.198.77       29   
30  241.136.198.77       30   
31  241.

In [120]:
#region run test
# Assuming ipv4 is the last column in your data (index -1)
import pandas as pd
import numpy as np
import sys
import ipaddress


# Drop the 'anomaly' column and prepare the data for Isolation Forest
features = df.drop(columns=['scan_id'])

features[['octet1', 'octet2', 'octet3', 'octet4']] = df['ipv4'].str.split('.', expand=True)
features[['octet1', 'octet2', 'octet3', 'octet4']] = features[['octet1', 'octet2', 'octet3', 'octet4']].astype(float)

features = features.drop(columns=['ipv4'])

# To use Isolation Forest, we need to convert the 'port' and 'service' columns into a suitable format
# For simplicity, we can flatten the 'port' and 'service' lists and create dummy variables
def flatten_columns(df, column):
    df_flattened = df.copy()
    for i in range(len(df)):
        for item in df.at[i, column]:
            df_flattened.at[i, f"{column}_{item}"] = 1
    df_flattened = df_flattened.fillna(0)
    return df_flattened.drop(columns=[column])

features_flat = flatten_columns(features, 'port')
features_flat = flatten_columns(features_flat, 'service')
ipv4_index = 0

model = CustomIsolationForest(ipv4_index=ipv4_index, n_estimators=2, random_state=0, df=df)
model.fit(features_flat)
labels = model.predict(features_flat)
scores = model.decision_function(features_flat)

print("LABELS ************************", labels)
print("SCORES ************************", scores)

#endregion

LABELS ************************ {'176.155.49.134': 1, '241.136.198.77': -1}
SCORES ************************ {'176.155.49.134': 0.025435796299047256, '241.136.198.77': -0.025435796299047242}


In [121]:
features

,port,service,anomaly,octet1,octet2,octet3,octet4
0,"[110, 5060, 587]","[POP3, SIP, SMTP]",0,176.0,155.0,49.0,134.0
1,"[10002, 5060, 110]","[HTTP, SIP, POP3]",0,176.0,155.0,49.0,134.0
2,"[5060, 10002, 110]","[SIP, HTTP, POP3]",0,176.0,155.0,49.0,134.0
3,[5060],[SIP],0,176.0,155.0,49.0,134.0
4,"[110, 587, 5060, 10002]","[POP3, SMTP, SIP, HTTP]",0,176.0,155.0,49.0,134.0
5,[110],[POP3],0,176.0,155.0,49.0,134.0
6,"[110, 587, 5060]","[POP3, SMTP, SIP]",0,176.0,155.0,49.0,134.0
7,"[587, 5060, 10002, 110]","[SMTP, SIP, HTTP, POP3]",0,176.0,155.0,49.0,134.0
8,"[5060, 110, 10002, 587]","[SIP, POP3, HTTP, SMTP]",0,176.0,155.0,49.0,134.0
9,"[5060, 10002, 110]","[SIP, HTTP, POP3]",0,176.0,155.0,49.0,134.0
